# **Data Preperation**

##**Goals**
## In this section, our objectives were to process the data received from the company's two clients to make it suitable for use in models. This involved several steps, including data cleaning, addressing missing values, adding necessary columns, removing unnecessary columns, and standardizing the data based on the provided Excel file from the company. Additionally, we created the urgency target variable and incorporated it into the dataset.

In [ ]:
!pip install pandas
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
from pylab import plot,show
import random
from sklearn.preprocessing import MinMaxScaler


In [ ]:
!pip install google.colab
from google.colab import drive
drive.mount('/content/d')

In [ ]:
file_path = '/content/d/MyDrive/final_project/Data_for_Urgency_Model.xlsx'

## **Standarization**

In [ ]:
#standardisation of priority coulmn

def standardize_priority(filepath):
    """Standardizes the values in the 'Priority' column of an Excel file.

    Parameters:
        filepath (str): The file path to the Excel file.

    Returns:
        pandas.DataFrame: The DataFrame with the 'Priority' column standardized.
    """

    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(filepath)


    # Convert all values in the priority column to lowercase
    df['Priority'] = df['Priority'].str.lower()


    # Define the priority mapping dictionary
    priority_map = {
        1: ['stat'],
        2: ['high'],
        3: ['routine']
    }


    # Iterate through the priority mapping dictionary and replace values in the priority column with their corresponding priority
    for priority, keywords in priority_map.items():
        for keyword in keywords:
            df['Priority'] = df['Priority'].replace(keyword, priority)


    # Convert the values in the priority column to integers
    df['Priority'] = pd.to_numeric(df['Priority'], errors='coerce')


    return df

    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(file_path)

    # Convert all values in the priority column to lowercase
    df['Priority'] = df['Priority'].str.lower()

    # Define the priority mapping dictionary
    priority_map = {
        1: ['stat'],
        2: ['high'],
        3: ['routine']
    }

    # Iterate through the priority mapping dictionary and replace values in the priority column with their corresponding priority
    for priority, keywords in priority_map.items():
        for keyword in keywords:
            df['Priority'] = df['Priority'].replace(keyword, priority)

    # Convert the values in the priority column to integers
    df['Priority'] = pd.to_numeric(df['Priority'], errors='coerce')

    return df


In [ ]:
df = standardize_priority(file_path)
df

## **Dealing with Missing Values**

In [ ]:
#Fill the blanck values in the column 'Patient Class' with NaN
df['Patient Class'].fillna(" ")
df['Patient Class']=df['Patient Class'].str.lower()
df['Patient Class'].unique()

In [ ]:
df['Exam Description'] = df['Exam Description'].fillna("unknown")

## **Removing columns**

In [ ]:
#Deleting unnecessary columns
def remove_columns(df, columns):
  return df.drop(columns=columns)
df=remove_columns(df, ['Exam Group', 'Referring Physician', 'Status', 'Ordered', 'First Image Acquired', 'Dictated', 'Transcribed', 'Reported', 'wRVU',
                    'Number of Images', 'Study_Count_2022'])

In [ ]:
file_path2 = '/content/d/MyDrive/final_project/Dictionary.xlsx'

## **Adding columns**

In [ ]:
#Creating "Body part" column
def add_body_part_column(df, file_path2, sheet_name):
    """
    Adds a 'body part' column to a DataFrame based on the values in an Excel file.

    Parameters:
    df (pandas.DataFrame): The DataFrame to add the 'body part' column to.
    file_path (str): The file path of the Excel file to read from.
    sheet_name (str): The name of the sheet in the Excel file to read from.

    Returns:
    pandas.DataFrame: The original DataFrame with the 'body part' column added.
    """
    # Read Excel file into a pandas DataFrame
    df_excel = pd.read_excel(file_path2, sheet_name=sheet_name)

    # Create a dictionary to map "Description contains" to "Body part" values
    sub_dict = {}
    for index, row in df_excel.iterrows():
        body_part = row['Body part']
        desc_contains = row['Description contains']
        sub_dict[desc_contains] = body_part

    # Create 'body_part' column based on 'Exam Description'
    body_part_list = []
    for index, row in df.iterrows():
        exam_desc = row['Exam Description']
        body_part = 'UNKNOWN'
        for desc_contains, part in sub_dict.items():
            if desc_contains.lower() in exam_desc.lower():
                body_part = part
                break
        body_part_list.append(body_part)

    df['Body Part'] = body_part_list
    df['Body Part'] = df['Body Part'].str.lower()

    return df


In [ ]:
df=add_body_part_column(df, file_path2, sheet_name='Description to body part')

In [ ]:
df

In [ ]:
df['Modality'].unique()

## **Standartization**

In [ ]:
#Creating standartization for "Modality" column
import pandas as pd

def standardize_modality(df, file_path2):
    """Standardizes the values in the 'Modality' column of a DataFrame using a dictionary of mappings.

    Parameters:
        df (pandas.DataFrame): The input DataFrame containing the 'Modality' column to be standardized.
        file_path2 (str): The file path to the Excel file containing the dictionary of mappings.

    Returns:
        pandas.DataFrame: The DataFrame with the 'Modality' column standardized according to the dictionary of mappings.
    """
    # Read the dictionary Excel file into a pandas DataFrame
    dictionary_df = pd.read_excel(file_path2, sheet_name='Atlantic&Avera modality')

    # Convert 'Atlantic&Avera Modality' and 'Modality new' columns to lowercase for case-insensitive matching
    dictionary_df['Atlantic&Avera Modality'] = dictionary_df['Atlantic&Avera Modality'].str.lower()
    dictionary_df['Modality new'] = dictionary_df['Modality new'].str.lower()

    # Convert 'Modality' column in the input DataFrame to lowercase
    df['Modality'] = df['Modality'].str.lower()

    # Apply the standardisation to the 'Modality' column in the input DataFrame
    for index, row in dictionary_df.iterrows():
        old_value = row['Atlantic&Avera Modality']
        new_value = row['Modality new']
        for index, row in df.iterrows():
          if row['Modality'] in old_value:
            df.at[index, 'Modality'] = new_value

    return df

In [ ]:
df=standardize_modality(df, file_path2)
df


In [ ]:
df['Modality'].unique()

## **Adding columns**

In [ ]:
# Adding a "subspecialty" column to the df
# Read Excel file into a pandas DataFrame
df_excel = pd.read_excel(file_path2, sheet_name='Body part and Mod to SubS')

# Create a dictionary to map (body part, modality) pairs to subspecialty values

sub_dict = {}
for index, row in df_excel.iterrows():
    subspecialty = row['Subspecialty']
    body_part = row['Body part new']
    modality = row['Modality']
    key = (modality, body_part)
    sub_dict[key] = subspecialty

# Create 'subspecialty_new' column based on 'body part' and 'modality'
sub_dict = {}
for index, row in df_excel.iterrows():
    subspecialty = row['Subspecialty']
    body_part = row['Body part new']
    modality = row['Modality']
    key = (modality.lower(), body_part.lower())
    sub_dict[key] = subspecialty

# Create 'subspecialty_new' column based on 'body part' and 'modality'
subspecialty_list = []
for index, row in df.iterrows():
    body_part = row['Body Part'].lower()
    modality = row['Modality'].lower()
    # Look for a matching (modality, body part) pair in the sub_dict
    key = (modality, body_part)
    if key in sub_dict:
        subspecialty_list.append(sub_dict[key])
    else:
        # If no exact match is found, try to find a match for any body part that contains the given body part substring
        possible_keys = [k for k in sub_dict.keys() if modality == k[0].lower() and body_part in k[1].lower()]
        if len(possible_keys) > 0:
            subspecialty_list.append(sub_dict[possible_keys[0]])
        else:
            subspecialty_list.append('unknown')

df['Subspecialty'] = subspecialty_list
df['Subspecialty']=df['Subspecialty'].str.lower()

# Print the updated DataFrame with the new 'subspecialty' column
df
print(df['Subspecialty'].value_counts()['unknown'])
print(df['Subspecialty'].unique())


In [ ]:
df


## **Visualizations**

In [ ]:
# Visualize the Subspecialty column
df['Subspecialty'].value_counts().plot(kind='bar')
plt.title("Distribution of Subspecialty")
plt.xlabel("Subspecialty")
plt.ylabel("Count")
plt.show()

In [ ]:
# Visualize the Modality column
df['Modality'].value_counts().plot(kind='bar')
plt.title("Distribution of Modality")
plt.xlabel("Modality")
plt.ylabel("Count")
plt.show()

In [ ]:
# Visualize the priority column
df['Priority'].value_counts().plot(kind='bar')
plt.title("Distribution of Priority")
plt.xlabel("Priority")
plt.ylabel("Count")
plt.show()

In [ ]:
# Visualize the priority column by Subspecialty
plt.figure(figsize=(10, 6))  # Adjust the figure size
# Group the data by 'Priority' and 'Subspecialty' columns and calculate the normalized value counts
grouped = df.groupby(['Priority'])['Subspecialty'].value_counts(normalize=True).unstack()
grouped.plot(kind='bar', cmap='tab20b')
plt.title('Priority by Subspecialty')
plt.xlabel('Priority')
plt.ylabel('%')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Subspecialty')
plt.show()


In [ ]:
# Visualize the Patient Class column
df['Patient Class'].value_counts().plot(kind='bar')
plt.title("Distribution of Patient Class")
plt.xlabel("Patient Class")
plt.ylabel("Count")
plt.show()

In [ ]:
# Visualize the Patient Class column by Subspecialty
plt.figure(figsize=(10, 6))  # Adjust the figure size
# Group the data by 'Priority' and 'Subspecialty' columns and calculate the normalized value counts
grouped = df.groupby(['Patient Class'])['Subspecialty'].value_counts(normalize=True).unstack()
grouped.plot(kind='bar', cmap='tab20b')
plt.title('Patient Class by Subspecialty')
plt.xlabel('Patient Class')
plt.ylabel('%')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Subspecialty')
plt.show()

In [ ]:
# Visualize the Patient Class column by Modality
plt.figure(figsize=(10, 6))  # Adjust the figure size
# Group the data by 'Priority' and 'Subspecialty' columns and calculate the normalized value counts
grouped = df.groupby(['Patient Class'])['Modality'].value_counts(normalize=True).unstack()
grouped.plot(kind='bar', cmap='tab20b')
plt.title('Patient Class by Modality')
plt.xlabel('Patient Class')
plt.ylabel('%')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Modality')
plt.show()

In [ ]:
#Visualising the priority column in division to subspecialties and customers
grouped = df.groupby(['Tenant DESC', 'Priority'])['Subspecialty'].value_counts().unstack().plot(kind='bar')
plt.title('Priority by Subspecialty')
plt.xlabel('Priority')
plt.ylabel('Count')
# Move the legend outside the graph
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.figure(figsize=(10, 10))  # Adjust the figure size if desired
plt.show()

In [ ]:
# Visualize the Body part column
plt.figure(figsize=(10, 9))
df['Body Part'].value_counts().plot(kind='bar')
plt.title("Distribution of Body part")
plt.xlabel("Body part")
plt.ylabel("Count")
plt.show()

In [ ]:
#Visualising the priority column in division to customers
grouped = df.groupby(['Priority'])['Tenant DESC'].value_counts().unstack().plot(kind='bar')
plt.title('Priority by Tenant DESC')
plt.xlabel('Priority')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualize the Patient Class column by Priority
plt.figure(figsize=(10, 6))  # Adjust the figure size
# Group the data by 'Patient Class' and 'Priority' columns and calculate the normalized value counts
grouped = df.groupby(['Patient Class'])['Priority'].value_counts(normalize=True).unstack()
grouped.plot(kind='bar', cmap='tab20b')
plt.title('Patient Class by Priority')
plt.xlabel('Patient Class')
plt.ylabel('%')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Priority')
plt.show()

In [ ]:
#Visualize the 'Patient Class' column in division to customer
grouped = df.groupby(['Patient Class'])['Tenant DESC'].value_counts().unstack().plot(kind='bar')
plt.title('Patient Class by Tenant DESC')
plt.xlabel('Patient Class')
plt.ylabel('Count')
plt.show()


In [ ]:
df

In [ ]:
df['Exam Description'] = df['Exam Description'].str.lower()
df

## **Dealing with Missing Values**

In [ ]:
df['Patient Class']=df['Patient Class'].fillna('unknown')

In [ ]:
df['Patient Class'].value_counts()

## **Reduction of missing values**

In [ ]:
# Update of 'Patient Class' column to 'outpatient' for rows where the 'Patient Class' is 'unknown' and the 'Modality' is 'mg'
for index, row in df.iterrows():
  if row['Patient Class'] == 'unknown' and row['Modality'] == 'mg':
    df.at[index, 'Patient Class'] ='outpatient'
print (df['Patient Class'].value_counts())

In [ ]:
# Update of 'Patient Class' value to 'outpatient' if the word 'outside' is present in the column Exam Description
for index, row in df.iterrows():
    words = row['Exam Description'].split()
    if "outside" in words:
        df.at[index, 'Patient Class'] = 'outpatient'

print(df['Patient Class'].value_counts())

## **Creation of target variable- Urgency score**

In [ ]:
df['Avera_Urgency'] = np.nan

In [ ]:
#create urgency score according to "Avera" rules, for all subjects

def get_urgency_score(df):
    """
    Calculates the urgency score based on priority and exam description for each record in a DataFrame.

    Parameters:
        df (pandas.DataFrame): The input DataFrame containing columns 'Priority', 'Patient Class', and 'Exam Description'.

    Returns:
        pandas.Series: The urgency scores as a Series.

    Urgency Score Calculation:
        - For 'outpatient' cases:
            - Priority 1: 22
            - Priority 2: 46
            - Priority 3: 58
        - For 'inpatient' cases:
            - Priority 1: 26
            - Priority 2: 42
            - Priority 3: 54
        - For 'emergency' cases: 10
        - If the exam description contains any of the keywords in the 'Exam_Desc' list (case-insensitive):
            - Urgency score is set to 1.
    """
    Exam_Desc = ['head', 'brain', 'stroke']
    df['Avera_Urgency'] = None  # initialize the column with None values

    for index, row in df.iterrows():
          exam_desc = row['Exam Description']
          if row['Patient Class'] == 'outpatient':
              if row['Priority'] == 1:
                  df.loc[index, 'Avera_Urgency'] = 22
              elif row['Priority'] == 2:
                  df.loc[index, 'Avera_Urgency'] = 46
              elif row['Priority'] == 3:
                  df.loc[index, 'Avera_Urgency'] = 58
          elif row['Patient Class'] == 'inpatient':
              if row['Priority'] == 1:
                  df.loc[index, 'Avera_Urgency'] = 26
              elif row['Priority'] == 2:
                  df.loc[index, 'Avera_Urgency'] = 42
              elif row['Priority'] == 3:
                  df.loc[index, 'Avera_Urgency'] = 54
          elif row['Patient Class'] == 'emergency':
            df.loc[index, 'Avera_Urgency'] = 10

            # Check if any part of the values in 'Exam_Desc' list appears in the exam description
            if any((keyword.lower() in exam_desc.lower()) for keyword in Exam_Desc):
                df.at[index, 'Avera_Urgency'] = 1

    return df['Avera_Urgency']


In [ ]:
df['Avera_Urgency'] = get_urgency_score(df)

In [ ]:
# replace NaN values with 0 in Avera_Urgency column
df['Avera_Urgency']=df['Avera_Urgency'].fillna(0).astype(int)

In [ ]:
df['Avera_Urgency'].value_counts()

In [ ]:
df['Atlantic_Urgency'] = np.nan

In [ ]:
#create urgency score according to "Atlantic" rules, for all subjects

def get_urgency_score2(df):
    """
    Calculates the urgency score based on priority, patient class, modality, and exam description for each record in a DataFrame,
    following the 'Atlantic' rules.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing columns 'Priority', 'Patient Class', 'Modality', and 'Exam Description'.

    Returns:
    pandas.Series: The urgency scores as a Series.

    Urgency Score Calculation:
        - For 'outpatient' cases:
            - Priority 1: 40
            - Priority 3: 90
        - For 'inpatient' cases:
            - Priority 1: 40
            - Priority 2: 50
            - Priority 3: 60
        - For 'emergency' cases: 30
        - If 'mg' is present in the modality: 91
        - If 'stroke' is present in the exam description (case-insensitive): 1
    """
    df['Atlantic_Urgency'] = None  # initialize the column with None values
    pattern = r'\bstroke\b'

    for index, row in df.iterrows():
        if row['Patient Class'] == 'outpatient':
          if row['Priority'] == 1:
            df.loc[index, 'Atlantic_Urgency'] = 40
          elif row['Priority'] == 3:
            df.loc[index, 'Atlantic_Urgency'] = 90
        elif row['Patient Class'] == 'inpatient':
          if row['Priority'] == 1:
            df.loc[index, 'Atlantic_Urgency'] = 40
          elif row['Priority'] == 2:
            df.loc[index, 'Atlantic_Urgency'] = 50
          elif row['Priority'] == 3:
            df.loc[index, 'Atlantic_Urgency'] = 60
        elif row['Patient Class'] == 'emergency':
          df.loc[index, 'Atlantic_Urgency'] = 30
        elif 'mg' in row['Modality']:
          df.loc[index, 'Atlantic_Urgency'] = 91
        if pd.Series(row['Exam Description']).str.contains('stroke', case=False).any():
          df.loc[index, 'Atlantic_Urgency'] = 1
    return df['Atlantic_Urgency']


In [ ]:
df['Atlantic_Urgency'] = get_urgency_score2(df)

In [ ]:
# replace NaN values with 0 in Atlantic_Urgency column
df['Atlantic_Urgency']=df['Atlantic_Urgency'].fillna(0).astype(int)

In [ ]:
df['Atlantic_Urgency'].value_counts(dropna=False)

In [ ]:
# Creating a new column of "Average_Urgency" based on Avera&Atlantic urgency, exsept for the mentioned scores.

scores = [0, 1, 20, 91]

def create_average(df):
    """
    Creates a new column 'Average_Urgency' in the DataFrame based on the 'Atlantic_Urgency' and 'Avera_Urgency' columns,
    excluding the mentioned scores.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing columns 'Atlantic_Urgency' and 'Avera_Urgency'.

    Returns:
     pandas.Series: The average urgency scores as a Series.

    Average Urgency Calculation:
        - If both 'Atlantic_Urgency' and 'Avera_Urgency' are not in the scores list:
            - The average urgency is calculated as the mean of 'Atlantic_Urgency' and 'Avera_Urgency' (skipping NaN values).
        - If either 'Atlantic_Urgency' or 'Avera_Urgency' is present in the scores list:
            - The average urgency is set to the corresponding score from the scores list.
        - The mentioned scores [0, 1, 20, 91] are excluded from the average calculation.
    """
    df['Average_Urgency'] = None
    for index, row in df.iterrows():
        atlantic = row['Atlantic_Urgency']
        avera = row['Avera_Urgency']
        if all(num not in [atlantic, avera] for num in scores):
            avg_urgency = pd.Series([atlantic, avera]).mean(skipna=True)
            df.loc[index, 'Average_Urgency'] = avg_urgency
        else:
            for num in scores:
                if num in [atlantic, avera]:
                    df.loc[index, 'Average_Urgency'] = num
                    break

    return df['Average_Urgency']


In [ ]:
df['Average_Urgency'] = create_average(df)

In [ ]:
df['Average_Urgency'] = df['Average_Urgency'].astype(int)

In [ ]:
df['Average_Urgency'].unique()

In [ ]:
df['Average_Urgency'].value_counts()

In [ ]:
df

In [ ]:
df.to_csv("Avera & Atlantic Data.csv")